In [ ]:
!pip install transformers==4.11.3.

     |████████████████████████████████| 2.9 MB 29.5 MB/s 
     |████████████████████████████████| 895 kB 58.6 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 62.0 MB/s 
     |████████████████████████████████| 3.3 MB 57.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import sys
from google.colab import drive
drive.mount('/content/gdrive/')
sys.path.append('/content/gdrive/MyDrive/data')

Mounted at /content/gdrive/


In [ ]:
SYSPATH = '/content/gdrive/MyDrive/data/'
import torch
import random
import numpy as np
config = {
    'train_file_path': SYSPATH + 'train.csv',
    'test_file_path': SYSPATH + 'test.csv',
    'train_val_ratio': 0.1,
    'model_path': SYSPATH + 'BERT_model',
    
    'batch_size': 16,
    'num_epochs': 2,
    'learning_rate': 2e-5,
    'logging_step': 500,
    'seed': 2021}

config['device']='cuda' if torch.cuda.is_available() else 'cpu'

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(config['seed'])

2021

In [ ]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
def read_data(config, tokenizer, mode='train'):
    df = pd.read_csv(config[mode + '_file_path'], sep=',')
    if mode == 'train':
        X_train, y_train = defaultdict(list), []
        X_val, y_val = defaultdict(list), []
        num_val = int(len(df) * config['train_val_ratio'])
    else:
        X_test, y_test = defaultdict(list), []

    for i, row in df.iterrows():
      # get label
        label=row[1] if mode == 'train' else 0
      # get sentence
        sentence = row[-1]

        # add_special_tokens  CLS、 SEP
        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, return_token_type_ids=True, return_attention_mask=True)

        if mode == 'train':
            if i < num_val:
                X_val['inputs_ids'].append(inputs['input_ids'])
                y_val.append(label)
                X_val['token_type_ids'].append(inputs['token_type_ids'])
                X_val['attention_mask'].append(inputs['attention_mask'])
                                    
            else:
                X_train['inputs_ids'].append(inputs['input_ids'])
                y_train.append(label)
                X_train['token_type_ids'].append(inputs['token_type_ids'])
                X_train['attention_mask'].append(inputs['attention_mask'])

        else:
            X_test['inputs_ids'].append(inputs['input_ids'])
            y_test.append(label) 
            X_test['token_type_ids'].append(inputs['token_type_ids'])
            X_test['attention_mask'].append(inputs['attention_mask'])
            
    # 返回
    if mode == 'train':
        label2id = {label: i for i, label in enumerate(np.unique(y_train))} 
        id2label = {i: label for label, i in label2id.items()} 
        y_train = torch.tensor([label2id[i] for i in y_train], dtype=torch.long)  
        y_val = torch.tensor([label2id[i] for i in y_val], dtype=torch.long)  
        return X_train, y_train, X_val, y_val, label2id, id2label
        
    else:
        y_test = torch.tensor(y_test, dtype=torch.long)
        return X_test, y_test

In [ ]:
from torch.utils.data import Dataset
class NEWSData(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y

    def __getitem__(self, idx):
        return {
            'inputs_ids' : self.x['inputs_ids'][idx],
            'label' : self.y[idx],
            'token_type_ids': self.x['token_type_ids'][idx],
            'attention_mask': self.x['attention_mask'][idx]
        }
    
    def __len__(self):
        return self.y.size(0)

In [ ]:
def collate_fn(examples):
    input_ids_list = []
    labels = []
    token_type_ids_list = []
    attention_mask_list = []

    for example in examples:
        input_ids_list.append(example['inputs_ids'])
        labels.append(example['label'])
        token_type_ids_list.append(example['token_type_ids'])
        attention_mask_list.append(example['attention_mask'])
    
    # get max_length
    max_length = max(len(input_ids) for input_ids in input_ids_list)
    input_ids_tensor = torch.zeros((len(labels), max_length), dtype=torch.long)
    token_type_ids_tensor = torch.zeros_like(input_ids_tensor)
    attention_mask_tensor = torch.zeros_like(input_ids_tensor)

    # insert values into the tensor
    for i, input_ids in enumerate(input_ids_list):
        input_ids_tensor[i, :len(input_ids)] = torch.tensor(input_ids, dtype=torch.long)
        token_type_ids_tensor[i, :len(input_ids)] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
        attention_mask_tensor[i, :len(input_ids)] = torch.tensor(attention_mask_list[i], dtype=torch.long)
    
    return{
        'input_ids' : input_ids_tensor,
        'labels' : torch.tensor(labels, dtype=torch.long),
        'token_type_ids': token_type_ids_tensor,
        'attention_mask': attention_mask_tensor
        }

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import DataLoader
def build_dataloader(config):
    # get bert pretrain
    tokenizer = BertTokenizer.from_pretrained(config['model_path'])
    X_train, y_train, X_val, y_val, label2id, id2label = read_data(config, tokenizer, mode='train')
    X_test, y_test = read_data(config, tokenizer, mode='test')

    train_dataset = NEWSData(X_train, y_train)
    val_dataset = NEWSData(X_val, y_val)
    test_dataset = NEWSData(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=True, collate_fn=collate_fn)
    val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)
    test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], num_workers=4, shuffle=False, collate_fn=collate_fn)

    return train_dataloader, val_dataloader, test_dataloader, id2label

train_dataloader, val_dataloader, test_dataloader, id2label = build_dataloader(config)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

def evaluation(config, model, val_dataloader):
    model.eval()
    preds = []
    labels = []
    val_loss = 0.
    val_iterator = tqdm(val_dataloader, desc='Evaluation', total=len(val_dataloader))

    with torch.no_grad():
        for batch in val_iterator:
            labels.append(batch['labels'])
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            loss, logits = model(**batch)[:2]

            val_loss += loss.item()
            preds.append(logits.argmax(dim=-1).detach().cpu())

    avg_val_loss = val_loss / len(val_dataloader)
    labels = torch.cat(labels, dim=0).numpy()
    preds = torch.cat(preds, dim=0).numpy()

    accuracy = accuracy_score(labels, preds)
    recall = recall_score(labels, preds, average='macro')
    precision = precision_score(labels, preds, average='macro')
    f1 = f1_score(labels, preds, average='macro')
    return [avg_val_loss, accuracy, recall, precision, f1]

In [ ]:
from transformers import BertConfig, BertForSequenceClassification
from transformers import AdamW
from tqdm import trange

def train(config, id2label, train_dataloader, val_dataloader):
    bert_config = BertConfig.from_pretrained(config['model_path'])
    bert_config.num_labels = len(id2label)
    model = BertForSequenceClassification.from_pretrained(config['model_path'], config=bert_config)

    # define optimizer
    optimizer = AdamW(model.parameters(), lr=config['learning_rate'])
    model.to(config['device'])
    global_steps = 0
    train_loss = 0.
    logging_loss = 0.

    
    for epoch in trange(config['num_epochs']):
      train_iterator = tqdm(train_dataloader, desc='Training', total=len(train_dataloader))
      model.train()
      for batch in train_dataloader:
        batch = {item: value.to(config['device']) for item, value in batch.items()}
        loss = model(**batch)[0]
        model.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        global_steps += 1

        if global_steps % config['logging_step'] == 0:
            print_train_loss = (train_loss - logging_loss) / config['logging_step']
            logging_loss = train_loss
            result = evaluation(config, model, val_dataloader)
            avg_val_loss, accuracy = result[0], result[1]
            print_log = f'>>> training loss: {print_train_loss:.4f}, valid loss: {avg_val_loss:.4f}, ' \
                        f'valid accuracy score: {accuracy:.4f}'
            print(print_log)
            model.train()

    return model

In [ ]:
model = train(config, id2label, train_dataloader, val_dataloader)

Some weights of the model checkpoint at /content/gdrive/MyDrive/data/BERT_model were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from t

>>> training loss: 1.6510, valid loss: 1.3553, valid accuracy score: 0.5480




Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Evaluation:   0%|          | 1/334 [00:00<01:07,  4.97it/s]

Evaluation:   1%|          | 4/334 [00:00<00:22, 14.57it/s]

Evaluation:   2%|▏         | 8/334 [00:00<00:14, 21.78it/s]

Evaluation:   3%|▎         | 11/334 [00:00<00:13, 23.87it/s]

Evaluation:   4%|▍         | 15/334 [00:00<00:12, 26.52it/s]

Evaluation:   6%|▌         | 19/334 [00:00<00:11, 28.14it/s]

Evaluation:   7%|▋         | 22/334 [00:00<00:11, 28.33it/s]

Evaluation:   7%|▋         | 25/334 [00:01<00:11, 27.87it/

>>> training loss: 1.3626, valid loss: 1.3106, valid accuracy score: 0.5560




Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Evaluation:   0%|          | 1/334 [00:00<01:09,  4.77it/s]

Evaluation:   1%|          | 4/334 [00:00<00:23, 14.03it/s]

Evaluation:   2%|▏         | 8/334 [00:00<00:15, 21.15it/s]

Evaluation:   3%|▎         | 11/334 [00:00<00:13, 23.47it/s]

Evaluation:   4%|▍         | 14/334 [00:00<00:12, 24.80it/s]

Evaluation:   5%|▌         | 18/334 [00:00<00:11, 27.14it/s]

Evaluation:   6%|▋         | 21/334 [00:00<00:11, 27.47it/s]

Evaluation:   7%|▋         | 24/334 [00:01<00:11, 27.30it/

...


Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

Evaluation: 100%|██████████| 334/334 [00:12<00:00, 26.25it/s]


>>> training loss: 1.0577, valid loss: 1.2486, valid accuracy score: 0.5543



Evaluation:   0%|          | 0/334 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))

Evaluation: 100%|██████████| 334/334 [00:12<00:00, 26.29it/s]


>>> training loss: 1.0398, valid loss: 1.2086, valid accuracy score: 0.5663


Training:   0%|          | 0/3002 [07:27<?, ?it/s]


In [ ]:
def predict(config, id2label, model, test_dataloader):
    test_iterator = tqdm(test_dataloader, desc='Predicting', total=len(test_dataloader))
    model.eval()
    test_preds = []
    with torch.no_grad():
        for batch in test_iterator:
            batch = {item: value.to(config['device']) for item, value in batch.items()}
            logits = model(**batch)[1]

            test_preds.append(logits.argmax(dim=-1).detach().cpu())
    test_preds = torch.cat(test_preds, dim=0).numpy()
    test_preds = [id2label[id_] for id_ in test_preds]
    test_df = pd.read_csv(config['test_file_path'], sep=',')
    test_df.insert(1, column='label', value=test_preds)
    test_df.drop(columns=['sentence'], inplace=True)
    
    return test_df

In [ ]:
prediction = predict(config, id2label, model, test_dataloader)
actual = pd.read_csv(SYSPATH  + 'solution.csv')

Predicting:   0%|          | 0/625 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Predicting: 100%|██████████| 625/625 [00:22<00:00, 27.71it/s]


In [ ]:
accuracy_score(actual['label'], prediction['label'])

0.5723

In [ ]:
recall_score(actual['label'], prediction['label'], average='weighted')

0.5723

In [ ]:
precision_score(actual['label'], prediction['label'], average='weighted')

0.5742307101239282

In [ ]:
f1_score(actual['label'], prediction['label'], average='weighted')

0.5722309621762148